In [24]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dir_name = "../data/evaluate/Bitcoin/"
senarios = os.listdir(dir_name)
folds = {}

for s in senarios:
    filenames = os.listdir(os.path.join(dir_name, s))
    filenames.sort()
    folds[s] = filenames

for s in senarios:
    df = pd.DataFrame()
    for f in folds[s]:
        _df = pd.read_parquet(os.path.join(dir_name, s, f), engine="pyarrow")
        df = df.append(_df)

    print(df.index[0], df.index[-1])

1514764860 1632182400
1514764860 1632182400
1514764860 1632182400
1514764860 1632182400
1514764860 1632182400


In [26]:
df.describe()

,Open,High,Low,Close,Volume,sma20,target,predict
count,1.956960e+06,1.956960e+06,1.956960e+06,1.956960e+06,1.956960e+06,1.956960e+06,1.956960e+06,1.956960e+06
mean,1.560868e+04,1.564936e+04,1.557561e+04,1.560868e+04,9.282028e+01,9.992031e-01,9.999969e-01,9.999975e-01
std,1.511957e+04,1.513020e+04,1.510443e+04,1.511960e+04,1.553879e+02,3.070064e-03,1.750992e-03,2.686173e-07
min,3.149657e+03,3.176910e+03,3.120000e+03,3.151545e+03,1.000000e-04,8.512298e-01,0.000000e+00,9.999975e-01
25%,6.849088e+03,6.866385e+03,6.836593e+03,6.849044e+03,3.018709e+01,9.987367e-01,1.000000e+00,9.999975e-01
50%,9.204972e+03,9.221200e+03,9.190260e+03,9.204930e+03,5.278533e+01,9.997243e-01,1.000000e+00,9.999975e-01
75%,1.345962e+04,1.351634e+04,1.339346e+04,1.346031e+04,9.765168e+01,1.000491e+00,1.000000e+00,9.999975e-01
max,6.480594e+04,6.490000e+04,6.467053e+04,6.480854e+04,1.040500e+04,1.164459e+00,1.000000e+00,1.000000e+00


In [30]:
import numpy as np

high_low = df["High"] - df["Low"]
high_close = np.abs(df["High"] - df["Close"].shift())
low_close = np.abs(df["Low"] - df["Close"].shift())
ranges = pd.concat([high_low, high_close, low_close], axis=1)
tr = np.max(ranges, axis=1)
atr = tr.rolling(14).sum() / 14
atr.head()

timestamp
1514764860   NaN
1514764920   NaN
1514764980   NaN
1514765040   NaN
1514765100   NaN
dtype: float64

In [43]:
pips = 0.5
limit_price_dist = atr * 0.5
# limit_price_dist = np.maximum(1, (limit_price_dist / pips).round().fillna(1)) * pips
limit_price_dist / pips

timestamp
1514764860           NaN
1514764920           NaN
1514764980           NaN
1514765040           NaN
1514765100           NaN
                 ...    
1632182160    106.354346
1632182220    104.562203
1632182280    102.780775
1632182340    112.655061
1632182400    108.289346
Length: 1956960, dtype: float64

In [46]:
limit_price_dist = np.maximum(1, (limit_price_dist / pips).round().fillna(1)) * pips
limit_price_dist

0.5


timestamp
1514764860     0.5
1514764920     0.5
1514764980     0.5
1514765040     0.5
1514765100     0.5
              ... 
1632182160    53.0
1632182220    52.5
1632182280    51.5
1632182340    56.5
1632182400    54.0
Length: 1956960, dtype: float64